In [2]:
# !pip install odfpy
# !pip install seaborn
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
from yaml import Loader
import numpy as np
import analyze_cells_energy as ace
import utils
from scipy.stats import ttest_ind, mannwhitneyu
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
root_directory = '/home/stella/Documents/Torino/projects/'
experiment = 'SexualImprinting'
experimental_group = 'BL_wf_USV_Bedding_wb'
data_directory = root_directory + experiment + '/' \
                + experimental_group + '/'
subjects = [name for name in os.listdir(data_directory) \
            if os.path.isdir(os.path.join(data_directory, name))]

In [4]:
# load query file where we added volumes for each area
volumes = ace.clean_volumes_database()

In [5]:
def load_subject(subject, data_directory, threshold):
    file_suffix = 'cells_' + str(threshold)
    df_mouse = pd.read_csv(data_directory + subject + '/' + file_suffix + '.csv')
    df_mouse = ace.reformat_df_mouse(df=df_mouse)
    
    return df_mouse

In [6]:
def print_results(experimental_groups, groups_df, area=None):
    for i, df in zip(experimental_groups.keys(), groups_df):
        print(i)
        if area==None:
            print(df.set_index('area').sum().to_string())
        else:
            print(df.set_index('area').loc[area].to_string())
        print(' ')

# Analysis on Black with father Whole Brain USVs and Bedding

In [7]:
# dict_results_across_mice = {subject: ace.calculate_cells_energy_per_level(
#     df_mouse=load_subject(subject=subject, 
#                           data_directory=data_directory,
#                           threshold=2500), 
#                                                                          vol=volumes, 
#                                                                          level=8) for subject in subjects}
# np.save('dict_results/dict_results_across_mice_BL_wf_USV_Bedding_wb.npy', dict_results_across_mice)

In [8]:
dict_results_across_mice = np.load('dict_results/dict_results_across_mice_BL_wf_USV_Bedding_wb.npy', 
                                   allow_pickle=True).item()

In [9]:
subjects = list(dict_results_across_mice.keys())

# List areas with null cell count

In [10]:
db_null_count = {subject: dict_results_across_mice[subject][
    dict_results_across_mice[subject].n_cells == 0] for subject in subjects}

# List areas with no volume

In [11]:
db_null_volume = {subject: dict_results_across_mice[subject][
    dict_results_across_mice[subject]['energy'].isnull()] for subject in subjects}

# Divide mice into experimental groups: FAM/UNFAM/CONTROL

In [12]:
experimental_groups = utils.divide_in_exp_groups(list_subjects=subjects,
                                                group_labels=['Control',
                                                       'USVC57',
                                                       'USVBALB',
                                                       'USVBALB_Bedding',
                                                       'USVC57_Bedding',
                                                       'Bedding'])
experimental_groups

{'Control': ['BL55Control',
  'BL79Control',
  'BL69Control',
  'BL62Control',
  'BL74Control'],
 'USVC57': ['BL81USVC57',
  'BL73USVC57',
  'BL67USVC57',
  'BL56USVC57',
  'BL63USVC57'],
 'USVBALB': ['BL61USVBALB',
  'BL80USVBALB',
  'BL68USVBALB',
  'BL57USVBALB',
  'BL75USVBALB'],
 'USVBALB_Bedding': ['BL78USVBALB_Bedding',
  'BL58USVBALB_Bedding',
  'BL71USVBALB_Bedding',
  'BL64USVBALB_Bedding',
  'BL84USVBALB_Bedding'],
 'USVC57_Bedding': ['BL77USVC57_Bedding',
  'BL59USVC57_Bedding',
  'BL70USVC57_Bedding',
  'BL82USVC57_Bedding',
  'BL66USVC57_Bedding'],
 'Bedding': ['BL60Bedding',
  'BL72Bedding',
  'BL83Bedding',
  'BL65Bedding',
  'BL76Bedding']}

# Calculate cell count across experimental groups

In [13]:
df_Control_cell_count, df_USVC57_cell_count, df_USVBALB_cell_count,\
df_USVBALB_Bedding_cell_count, df_USVC57_Bedding_cell_count, df_Bedding_cell_count = \
ace.calculate_value_across_groups(experimental_groups=experimental_groups, 
                              dict_results_across_mice=dict_results_across_mice, 
                              value='n_cells')
groups_df = [df_Control_cell_count, df_USVC57_cell_count, df_USVBALB_cell_count,\
df_USVBALB_Bedding_cell_count, df_USVC57_Bedding_cell_count, df_Bedding_cell_count]

In [14]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df)

Control
BL55Control    217841
BL79Control    137308
BL69Control    127369
BL62Control     73044
BL74Control    136898
 
USVC57
BL81USVC57     79021
BL73USVC57    155085
BL67USVC57    150832
BL56USVC57    138517
BL63USVC57    173128
 
USVBALB
BL61USVBALB     65856
BL80USVBALB    101695
BL68USVBALB    188648
BL57USVBALB    149626
BL75USVBALB    148906
 
USVBALB_Bedding
BL78USVBALB_Bedding    150121
BL58USVBALB_Bedding    180282
BL71USVBALB_Bedding    157923
BL64USVBALB_Bedding    159550
BL84USVBALB_Bedding    171060
 
USVC57_Bedding
BL77USVC57_Bedding    129170
BL59USVC57_Bedding    276063
BL70USVC57_Bedding    175941
BL82USVC57_Bedding    198354
BL66USVC57_Bedding    134469
 
Bedding
BL60Bedding    183583
BL72Bedding    228225
BL83Bedding     93141
BL65Bedding     93964
BL76Bedding     24701
 


In [15]:
df_USVC57_Bedding_cell_count.set_index('area').sum()

BL77USVC57_Bedding    129170
BL59USVC57_Bedding    276063
BL70USVC57_Bedding    175941
BL82USVC57_Bedding    198354
BL66USVC57_Bedding    134469
dtype: int64

In [16]:
df_Bedding_cell_count.set_index('area').sum()

BL60Bedding    183583
BL72Bedding    228225
BL83Bedding     93141
BL65Bedding     93964
BL76Bedding     24701
dtype: int64

# Look at cell count per area

In [17]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Inferior colliculus')

Control
BL55Control    1799
BL79Control     701
BL69Control     515
BL62Control     626
BL74Control     418
 
USVC57
BL81USVC57     542
BL73USVC57     985
BL67USVC57    1070
BL56USVC57    1013
BL63USVC57     979
 
USVBALB
BL61USVBALB     525
BL80USVBALB     650
BL68USVBALB    1336
BL57USVBALB     553
BL75USVBALB     640
 
USVBALB_Bedding
BL78USVBALB_Bedding     699
BL58USVBALB_Bedding     479
BL71USVBALB_Bedding     687
BL64USVBALB_Bedding    1193
BL84USVBALB_Bedding     805
 
USVC57_Bedding
BL77USVC57_Bedding     869
BL59USVC57_Bedding    2096
BL70USVC57_Bedding     927
BL82USVC57_Bedding     544
BL66USVC57_Bedding    1040
 
Bedding
BL60Bedding    1172
BL72Bedding     924
BL83Bedding     439
BL65Bedding     896
BL76Bedding     212
 


## AUDITORY

In [18]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Supramammillary nucleus')

Control
BL55Control    197
BL79Control    380
BL69Control     68
BL62Control     92
BL74Control     86
 
USVC57
BL81USVC57     72
BL73USVC57    128
BL67USVC57     24
BL56USVC57    178
BL63USVC57    236
 
USVBALB
BL61USVBALB    111
BL80USVBALB    111
BL68USVBALB    109
BL57USVBALB    120
BL75USVBALB     68
 
USVBALB_Bedding
BL78USVBALB_Bedding    146
BL58USVBALB_Bedding    199
BL71USVBALB_Bedding    222
BL64USVBALB_Bedding    216
BL84USVBALB_Bedding    186
 
USVC57_Bedding
BL77USVC57_Bedding    190
BL59USVC57_Bedding    277
BL70USVC57_Bedding    132
BL82USVC57_Bedding    165
BL66USVC57_Bedding    195
 
Bedding
BL60Bedding    112
BL72Bedding    124
BL83Bedding    176
BL65Bedding    202
BL76Bedding     87
 


In [19]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Primary auditory area')

Control
BL55Control    483
BL79Control    347
BL69Control    245
BL62Control    189
BL74Control    216
 
USVC57
BL81USVC57    122
BL73USVC57    303
BL67USVC57    427
BL56USVC57    434
BL63USVC57    530
 
USVBALB
BL61USVBALB    180
BL80USVBALB    243
BL68USVBALB    321
BL57USVBALB    672
BL75USVBALB    472
 
USVBALB_Bedding
BL78USVBALB_Bedding    188
BL58USVBALB_Bedding    320
BL71USVBALB_Bedding    196
BL64USVBALB_Bedding    290
BL84USVBALB_Bedding    371
 
USVC57_Bedding
BL77USVC57_Bedding    176
BL59USVC57_Bedding    907
BL70USVC57_Bedding    432
BL82USVC57_Bedding    336
BL66USVC57_Bedding    236
 
Bedding
BL60Bedding    247
BL72Bedding    817
BL83Bedding     72
BL65Bedding     69
BL76Bedding     13
 


In [20]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Dorsal auditory area')

Control
BL55Control    242
BL79Control    247
BL69Control    103
BL62Control     88
BL74Control    110
 
USVC57
BL81USVC57     98
BL73USVC57    211
BL67USVC57    217
BL56USVC57    318
BL63USVC57    187
 
USVBALB
BL61USVBALB     30
BL80USVBALB    115
BL68USVBALB    136
BL57USVBALB    440
BL75USVBALB    309
 
USVBALB_Bedding
BL78USVBALB_Bedding    127
BL58USVBALB_Bedding    198
BL71USVBALB_Bedding    169
BL64USVBALB_Bedding    107
BL84USVBALB_Bedding    163
 
USVC57_Bedding
BL77USVC57_Bedding     93
BL59USVC57_Bedding    655
BL70USVC57_Bedding    320
BL82USVC57_Bedding    288
BL66USVC57_Bedding    106
 
Bedding
BL60Bedding    145
BL72Bedding    588
BL83Bedding     29
BL65Bedding     25
BL76Bedding      2
 


In [21]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Ventral auditory area')

Control
BL55Control    695
BL79Control    415
BL69Control    305
BL62Control    181
BL74Control    288
 
USVC57
BL81USVC57    259
BL73USVC57    451
BL67USVC57    498
BL56USVC57    527
BL63USVC57    655
 
USVBALB
BL61USVBALB    182
BL80USVBALB    332
BL68USVBALB    577
BL57USVBALB    685
BL75USVBALB    533
 
USVBALB_Bedding
BL78USVBALB_Bedding    236
BL58USVBALB_Bedding    327
BL71USVBALB_Bedding    286
BL64USVBALB_Bedding    556
BL84USVBALB_Bedding    609
 
USVC57_Bedding
BL77USVC57_Bedding     302
BL59USVC57_Bedding    1091
BL70USVC57_Bedding     514
BL82USVC57_Bedding     521
BL66USVC57_Bedding     365
 
Bedding
BL60Bedding    359
BL72Bedding    952
BL83Bedding    118
BL65Bedding    116
BL76Bedding     10
 


## VENTRAL

In [22]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Ventral premammillary nucleus')

Control
BL55Control    13
BL79Control    43
BL69Control     1
BL62Control     7
BL74Control     3
 
USVC57
BL81USVC57     3
BL73USVC57    30
BL67USVC57     3
BL56USVC57    11
BL63USVC57    13
 
USVBALB
BL61USVBALB    13
BL80USVBALB    18
BL68USVBALB     6
BL57USVBALB     4
BL75USVBALB     5
 
USVBALB_Bedding
BL78USVBALB_Bedding    354
BL58USVBALB_Bedding    421
BL71USVBALB_Bedding    395
BL64USVBALB_Bedding    305
BL84USVBALB_Bedding    237
 
USVC57_Bedding
BL77USVC57_Bedding    377
BL59USVC57_Bedding    399
BL70USVC57_Bedding    389
BL82USVC57_Bedding    313
BL66USVC57_Bedding    453
 
Bedding
BL60Bedding    332
BL72Bedding    406
BL83Bedding    306
BL65Bedding    353
BL76Bedding     43
 


## MAIN

In [23]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Main olfactory bulb')

Control
BL55Control    7370
BL79Control    5444
BL69Control    4894
BL62Control    5730
BL74Control    6164
 
USVC57
BL81USVC57     1839
BL73USVC57    12464
BL67USVC57    12178
BL56USVC57     8581
BL63USVC57    18060
 
USVBALB
BL61USVBALB    3000
BL80USVBALB    6390
BL68USVBALB    9323
BL57USVBALB    5431
BL75USVBALB    6255
 
USVBALB_Bedding
BL78USVBALB_Bedding     7214
BL58USVBALB_Bedding    15102
BL71USVBALB_Bedding    16596
BL64USVBALB_Bedding    12797
BL84USVBALB_Bedding    12387
 
USVC57_Bedding
BL77USVC57_Bedding     5799
BL59USVC57_Bedding    21364
BL70USVC57_Bedding    10252
BL82USVC57_Bedding    12435
BL66USVC57_Bedding    10572
 
Bedding
BL60Bedding    14516
BL72Bedding    16254
BL83Bedding     8800
BL65Bedding     2688
BL76Bedding     2202
 


## AOB

In [24]:
print_results(experimental_groups=experimental_groups,
              groups_df=groups_df,
             area='Accessory olfactory bulb')

Control
BL55Control    164
BL79Control    131
BL69Control    142
BL62Control     88
BL74Control     83
 
USVC57
BL81USVC57     27
BL73USVC57     80
BL67USVC57     83
BL56USVC57    124
BL63USVC57    160
 
USVBALB
BL61USVBALB     63
BL80USVBALB     44
BL68USVBALB     51
BL57USVBALB    184
BL75USVBALB     81
 
USVBALB_Bedding
BL78USVBALB_Bedding    1339
BL58USVBALB_Bedding    2065
BL71USVBALB_Bedding    1989
BL64USVBALB_Bedding    2902
BL84USVBALB_Bedding    1636
 
USVC57_Bedding
BL77USVC57_Bedding    1900
BL59USVC57_Bedding    2633
BL70USVC57_Bedding    2718
BL82USVC57_Bedding    2442
BL66USVC57_Bedding    2362
 
Bedding
BL60Bedding    1695
BL72Bedding    1936
BL83Bedding    1149
BL65Bedding     696
BL76Bedding     654
 


## Calculate t-tests across groups per area

In [25]:
df_ttest_ncells = ace.test_across_groups([df_control_cell_count,df_fam_cell_count,df_unfam_cell_count])

NameError: name 'df_control_cell_count' is not defined

In [ ]:
# remove rows corresponding to area that have zero cell count across all experimental groups
df_ttest_not_null_ncells = df_ttest_ncells.dropna(axis=0,
                                   how='all',
                                   subset=['pval_Control_vs_Fam',
                                          'pval_Control_vs_Unfam',
                                          'pval_Fam_vs_Unfam'])

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_ncells.sort_values(by='pval_Control_vs_Fam')[['area', 'pval_Control_vs_Fam']][0:35].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs unfam and pick the first 20
df_ttest_not_null_ncells.sort_values(by='pval_Control_vs_Fam')[['area', 'pval_Control_vs_Fam']].set_index('area').loc['Ventral premammillary nucleus']

In [ ]:
# sort dataframe pvalues of control vs unfam and pick the first 20
df_ttest_not_null_ncells.sort_values(by='pval_Control_vs_Unfam')[['area', 'pval_Control_vs_Unfam']][0:15].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs unfam and pick the first 20
df_ttest_not_null_ncells.sort_values(by='pval_Control_vs_Unfam')[['area', 'pval_Control_vs_Unfam']].set_index('area').loc['Supramammillary nucleus']

In [ ]:
# sort dataframe pvalues of fam vs unfam and pick the first 20
df_ttest_not_null_ncells.sort_values(by='pval_Fam_vs_Unfam')[['area', 'pval_Fam_vs_Unfam']][0:40].set_index('area')

In [ ]:
df_ttest_not_null_ncells.sort_values(by='pval_Fam_vs_Unfam')[['area', 'pval_Fam_vs_Unfam']].set_index('area').loc['Supramammillary nucleus']

## Calculate cross corr across groups per area on cell count

In [ ]:
corr_matrix_control_ncells = ace.cross_corr(df_control_cell_count)

In [ ]:
corr_matrix_fam_ncells = ace.cross_corr(df_fam_cell_count)

In [ ]:
corr_matrix_unfam_ncells = ace.cross_corr(df_unfam_cell_count)

## Plot cross corr matrix across groups per area on cell count

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_control_ncells,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('CONTROL')

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_fam_ncells,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('FAM')

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_unfam_ncells,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('UNFAM')

# Calculate energy across experimental groups

In [ ]:
df_control_energy, df_fam_energy, df_unfam_energy = \
ace.calculate_value_across_groups(experimental_groups, dict_results_across_mice, value='energy')

## T-test on energy across groups

In [ ]:
df_ttest_energy = ace.test_across_groups([df_control_energy,df_fam_energy,df_unfam_energy])

In [ ]:
# remove rows corresponding to area that have zero cell count across all experimental groups
df_ttest_not_null_energy = df_ttest_energy.dropna(axis=0,
                                   how='all',
                                   subset=['pval_Control_vs_Fam',
                                          'pval_Control_vs_Unfam',
                                          'pval_Fam_vs_Unfam'])

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_energy.sort_values(by='pval_Control_vs_Fam')[['area', 'pval_Control_vs_Fam']][0:10].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_energy.sort_values(by='pval_Control_vs_Unfam')[['area', 'pval_Control_vs_Unfam']][0:15].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_energy.sort_values(by='pval_Fam_vs_Unfam')[['area', 'pval_Fam_vs_Unfam']][0:20].set_index('area')

In [ ]:
corr_matrix_control_energy = ace.cross_corr(df_control_energy)

In [ ]:
corr_matrix_fam_energy = ace.cross_corr(df_fam_energy)

In [ ]:
corr_matrix_unfam_energy = ace.cross_corr(df_unfam_energy)

## Plot cross corr matrix across groups per area on energy

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_control_energy,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('CONTROL')

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_fam_energy,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('FAM')

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix_unfam_energy,
    cmap=sns.diverging_palette(220, 10, as_cmap=True),
    vmin=-1.0, vmax=1.0,
    square=True, ax=ax)
ax.set_title('UNFAM')

# Calculate relative density

In [ ]:
df_control_rd, df_fam_rd, df_unfam_rd = \
ace.calculate_value_across_groups(experimental_groups, dict_results_across_mice, value='relative_density')

In [ ]:
df_ttest_rd = ace.test_across_groups([df_control_rd,df_fam_rd,df_unfam_rd])

In [ ]:
# remove rows corresponding to area that have zero cell count across all experimental groups
df_ttest_not_null_rd = df_ttest_rd.dropna(axis=0,
                                   how='all',
                                   subset=['pval_Control_vs_Fam',
                                          'pval_Control_vs_Unfam',
                                          'pval_Fam_vs_Unfam'])

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_rd.sort_values(by='pval_Control_vs_Fam')[['area', 'pval_Control_vs_Fam']][0:20].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_rd.sort_values(by='pval_Control_vs_Unfam')[['area', 'pval_Control_vs_Unfam']][0:20].set_index('area')

In [ ]:
# sort dataframe pvalues of control vs fam and pick the first 20
df_ttest_not_null_rd.sort_values(by='pval_Fam_vs_Unfam')[['area', 'pval_Fam_vs_Unfam']][0:20].set_index('area')